# MASA-OA Tracker - Benchmark Evaluation Setup

This notebook sets up and runs the MASA-OA tracker on MOT benchmarks using Google Colab's GPU.

## Prerequisites
- Enable GPU runtime: Runtime → Change runtime type → GPU (T4 or better)
- Your MASA-OA repository (we'll clone from GitHub or upload from Drive)

## Step 1: Check GPU Availability

In [ ]:
print("Hello")

In [1]:
!nvidia-smi

## Step 2: Setup Repository

Choose one of the following methods:

### Method A: Clone from GitHub (if your repo is on GitHub)

In [ ]:
# Clone your repository
!git clone https://github.com/YOUR_USERNAME/MASA-OA.git
%cd MASA-OA

### Method B: Mount Google Drive and copy from there

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy your MASA-OA folder from Drive (adjust the path)
!cp -r /content/drive/MyDrive/MASA-OA /content/
%cd /content/MASA-OA

### Method C: Upload as ZIP file

In [ ]:
# Upload your MASA-OA.zip file using the file upload widget
from google.colab import files
uploaded = files.upload()  # This will open a file picker

# Extract the ZIP
!unzip MASA-OA.zip
%cd MASA-OA

## Step 3: Verify Repository Structure

In [ ]:
# Check directory structure
!ls -la
!echo "\n=== Benchmark sequences ==="
!ls benchmarks/MOT15/

## Step 4: Install Dependencies

Install all required packages for MASA-OA tracker.

In [ ]:
# Install PyTorch with CUDA support (if not already installed)
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# Install mmcv and other dependencies
# Note: Adjust versions based on your requirements.txt or setup.py
!pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"
!mim install "mmdet>=3.0.0"

In [ ]:
# Install additional dependencies
!pip install tqdm
!pip install opencv-python
!pip install numpy
!pip install scipy
!pip install matplotlib

# If you have a requirements.txt, use it instead:
# !pip install -r requirements.txt

## Step 5: Download MASA Model Weights

Download the pretrained MASA-R50 weights.

In [ ]:
# Create directory for model weights
!mkdir -p saved_models/masa_models

# Download MASA-R50 weights
!wget -O saved_models/masa_models/masa_r50.pth \
    https://huggingface.co/dereksiyuanli/masa/resolve/main/masa_r50.pth

# Verify download
!ls -lh saved_models/masa_models/

## Step 6: Test Single Sequence

Run the tracker on a single sequence to verify everything works.

In [ ]:
# Test with Venice-2 sequence
!python run_benchmark.py \
    --benchmark benchmarks/MOT15 \
    --sequences Venice-2 \
    --masa-config configs/masa-one/masa_r50_plug_and_play.py \
    --masa-checkpoint saved_models/masa_models/masa_r50.pth \
    --device cuda:0 \
    --show-fps

## Step 7: Process All MOT15 Sequences

Run the tracker on all sequences in the benchmark.

In [ ]:
# Process all MOT15 sequences
!python run_benchmark.py \
    --benchmark benchmarks/MOT15 \
    --masa-config configs/masa-one/masa_r50_plug_and_play.py \
    --masa-checkpoint saved_models/masa_models/masa_r50.pth \
    --device cuda:0 \
    --show-fps \
    --continue-on-error

## Step 8 (Optional): Generate Ground Truth Visualization Videos

In [ ]:
# Process specific sequences with GT visualization
!python run_benchmark.py \
    --benchmark benchmarks/MOT15 \
    --sequences Venice-2 KITTI-13 \
    --masa-config configs/masa-one/masa_r50_plug_and_play.py \
    --masa-checkpoint saved_models/masa_models/masa_r50.pth \
    --device cuda:0 \
    --generate-gt-video

## Step 9: Check Results

In [ ]:
# List all generated tracking results
!find evaluation/MOT15 -name "*.txt" -type f

# Show summary of results
!echo "\n=== Result files generated ==="
!ls -lh evaluation/MOT15/*/*.txt

In [ ]:
# Preview a tracking result file
!echo "Preview of Venice-2 tracking results:"
!head -20 evaluation/MOT15/Venice-2/Venice-2_masa-hung-kf-OcclusionAware.txt

## Step 10: Download Results

Download the tracking results to your local machine or save to Google Drive.

In [ ]:
# Option A: Zip and download all results
!zip -r tracking_results.zip evaluation/

# Download the zip file
from google.colab import files
files.download('tracking_results.zip')

In [ ]:
# Option B: Copy results to Google Drive
!cp -r evaluation /content/drive/MyDrive/MASA-OA-Results/
print("Results saved to Google Drive!")

## Troubleshooting

### Common Issues

1. **Out of Memory (OOM) errors:**
   - Use `--fp16` flag for FP16 mode
   - Process sequences one at a time
   - Clear GPU memory between runs:
   ```python
   import torch
   torch.cuda.empty_cache()
   ```

2. **Missing dependencies:**
   - Check if you have a requirements.txt and install it
   - Install missing packages individually

3. **CUDA errors:**
   - Verify GPU is enabled: Runtime → Change runtime type → GPU
   - Restart runtime if needed

4. **File not found errors:**
   - Verify benchmarks directory structure
   - Check that video and gt.txt files exist

## Advanced: Process Specific Sequences with Custom Settings

In [ ]:
# Define sequences to process
sequences = ['Venice-2', 'KITTI-13', 'ETH-Sunnyday']

# Process each with custom settings
for seq in sequences:
    print(f"\n{'='*80}")
    print(f"Processing: {seq}")
    print(f"{'='*80}\n")
    
    !python run_benchmark.py \
        --benchmark benchmarks/MOT15 \
        --sequences {seq} \
        --masa-config configs/masa-one/masa_r50_plug_and_play.py \
        --masa-checkpoint saved_models/masa_models/masa_r50.pth \
        --device cuda:0 \
        --show-fps \
        --fp16
    
    # Clear GPU cache between sequences
    import torch
    torch.cuda.empty_cache()
    print(f"Completed: {seq}\n")

## Monitor GPU Usage

In [ ]:
# Check GPU memory usage
!nvidia-smi --query-gpu=memory.used,memory.total,utilization.gpu --format=csv